## The aim of this notebook is creating a pipeline to assigne to each client the Estimate in the incremental sales uplift caused by an invitation (i.e., the causal effect of the invitation).

- Treatment: Invitation to an event (client_is_invited in Actions).
- Outcome: Post-event sales (e.g., gross_amount_euro in Transactions within a defined window after the event).

In [2]:
import pandas as pd

In [4]:
# import Data
actions = pd.read_excel("XHEC-Eleven-Project/data/actions.xlsx")
clients = pd.read_excel("XHEC-Eleven-Project/data/clients.xlsx")
transactions = pd.read_excel("XHEC-Eleven-Project/data/transactions.xlsx")